# Exercise 13 - Production data to SQL DB

## Learning Objectives

* Explore sqlalchemy and pandas libraries for DB interaction
* Create DB
* Create Empty tables
* Populate well metadata 
* Populate Production table from Pandas

## Import libraries

In [13]:
import pandas as pd
import sqlalchemy as db
import os
import credentials
from sqlalchemy import Table, Column, String, Integer, Float
from ipywidgets import *
import matplotlib.pyplot as plt

## Import and explore Volve Data

Volve is a North Sea field operated by Equinor, already decommissioned. Equinor has made their data public available, raw data and interpretation. We have extracted the production data and some of the well metadata for the purpose of this exercise.

Source Data: https://www.equinor.com/en/how-and-why/digitalisation-in-our-dna/volve-field-data-village-download.html

### Import Production Data

In [ ]:
# Let's load the data into the notebook
file = "Volve production data.xlsx"
if os.path.exists(file):
    df_prod = pd.read_excel(file)
else:
    fileurl = "https://github.com/orkahub/PEG_Python/raw/master/Data/Volve/Production_data/Volve%20production%20data.xlsx"
    df_prod = pd.read_excel(fileurl)

In [6]:
df_prod.head(5)

,DATEPRD,WELL_BORE_CODE,NPD_WELL_BORE_CODE,NPD_WELL_BORE_NAME,NPD_FIELD_CODE,NPD_FIELD_NAME,NPD_FACILITY_CODE,NPD_FACILITY_NAME,ON_STREAM_HRS,AVG_DOWNHOLE_PRESSURE,...,AVG_CHOKE_UOM,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,BORE_WAT_VOL,BORE_WI_VOL,FLOW_KIND,WELL_TYPE
0,2014-04-07,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,0.00000,...,%,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,production,WI
1,2014-04-08,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,NaN,...,%,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,production,OP
2,2014-04-09,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,NaN,...,%,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,production,OP
3,2014-04-10,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,NaN,...,%,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,production,OP
4,2014-04-11,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,310.37614,...,%,33.09788,10.47992,33.07195,0.0,0.0,0.0,NaN,production,OP


In [8]:
column_names = {
    'NPD_WELL_BORE_NAME': 'well', 
    'DATEPRD': 'dates', 
    'BORE_OIL_VOL': 'oilvol', 
    'BORE_WAT_VOL': 'watervol', 
    'BORE_GAS_VOL': 'gasvol',
    'BORE_WI_VOL': 'waterinj',
    'AVG_DOWNHOLE_PRESSURE': 'bhp',
    'AVG_WHP_P': 'thp',
    'AVG_DOWNHOLE_TEMPERATURE': 'bht'
}
df_prod.rename(columns=column_names, inplace=True)

In [9]:
df_prod.head(5)

,dates,WELL_BORE_CODE,NPD_WELL_BORE_CODE,well,NPD_FIELD_CODE,NPD_FIELD_NAME,NPD_FACILITY_CODE,NPD_FACILITY_NAME,ON_STREAM_HRS,bhp,...,AVG_CHOKE_UOM,thp,AVG_WHT_P,DP_CHOKE_SIZE,oilvol,gasvol,watervol,waterinj,FLOW_KIND,WELL_TYPE
0,2014-04-07,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,0.00000,...,%,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,production,WI
1,2014-04-08,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,NaN,...,%,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,production,OP
2,2014-04-09,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,NaN,...,%,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,production,OP
3,2014-04-10,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,NaN,...,%,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,production,OP
4,2014-04-11,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,310.37614,...,%,33.09788,10.47992,33.07195,0.0,0.0,0.0,NaN,production,OP


## Setup the connection to the DB and Fill with Data

We will be working with an Azure cloud DataBase, this will be available for 5 days.

### Basic connection settings

In [15]:
## Create the connection to my DB
server = credentials.server
user = credentials.user
password = credentials.password
sql_flavour = 'postgresql://'
#engine_string = sql_flavour + user + ":" + password + "@" + server +  ":5432/" + db_name
engine_string = sql_flavour + user + ":" + password + "@" + server +  ":5432/postgres" 
engine_string

'postgresql://TrainingPythonAdmin@pythontraining:Python2021O&G@pythontraining.postgres.database.azure.com:5432/postgres'

In [16]:
server

'pythontraining.postgres.database.azure.com'

### Creating the connection

In [19]:
engine = db.create_engine(engine_string)

In [21]:
connection = engine.connect()

### Create your first Database

In [25]:
db_name = 'Volve_Instructor_AP2' #Change the following name to your own:
connection.execute('commit')

In [26]:
query = "CREATE DATABASE "+db_name
connection.execute(query)

### Create and populate well metadata

In [27]:
meta = db.MetaData()
wellmeta = Table(
    'wellmeta', meta,
    Column('id', Integer, primary_key=True),
    Column('well', String),
    Column('latitude', Float),
    Column('longitude', Float)
)
meta.create_all(engine)

In [29]:
wellnames= ['15/9-F-1 C', '15/9-F-11', '15/9-F-12', '15/9-F-14', '15/9-F-15 D', '15/9-F-4', '15/9-F-5']
# Top formation location
lat = [58.450073611111115, 58.44036305555555, 58.43840611111111, 58.442518611111105, 58.443219166666665,
 58.433746944444444, 58.440733611111106]
long = [1.8963722222222221, 1.9132308333333332, 1.8924699999999999, 1.8920077777777777, 1.9148333333333332,
 1.8980691666666667, 1.9121891666666666]

In [30]:
for i, well in enumerate(wellnames):
    insertquery = wellmeta.insert().values(id=i, well=well, latitude=lat[i], longitude=long[i])

### Create and populate Production Data

## Import Data from DB into Jupyter

In [8]:
def CustomQuery(table, quantities, well):

    return df_temp

In [ ]:
CustomQuery('production', 'dates, bhp ', '15/9-F-1 C')

In [10]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [ ]:
@interact
def PlotWellData():
    plt.plot()